# Enshitificador de Datos - Dataset de Criptomonedas

**Objetivo** Ensuciar un dataset mediante código.

## Tipos de Errores Introducidos (12+):

1. **Datos Faltantes** - Valores NaN aleatorios
2. **Filas Duplicadas** - Duplicados exactos y parciales
3. **Valores Atípicos** - Valores extremos en columnas numéricas
4. **Inconsistencias de Formato** - Formatos numéricos mixtos
5. **Errores Tipográficos** - Errores de ortografía en nombres
6. **Categorías Extra** - Valores inusuales en columnas categóricas
7. **Tipos de Datos Incorrectos** - Números almacenados como texto
8. **Codificación No-UTF-8** - Caracteres especiales
9. **Encabezados Incorrectos** - Nombres de columnas modificados
10. **Puntuación Extra** - Símbolos de moneda, comas en números
11. **Capitalización Inconsistente** - Mayúsculas/minúsculas mixtas
12. **Problemas de Espacios** - Espacios al inicio/final

In [ ]:
import pandas as pd
import numpy as np
import random
import string

# Establecer semilla aleatoria para reproducibilidad
np.random.seed(42)
random.seed(42)

print("¡Librerías importadas exitosamente!")

¡Librerías importadas exitosamente!


## 1. Cargar Dataset Limpio

In [ ]:
# Cargar el dataset limpio
df = pd.read_csv('source_clean_dataset.csv')

print(f"Dimensiones del dataset original: {df.shape}")
print(f"\nColumnas originales: {df.columns.tolist()}")
print(f"\nPrimeras 5 filas:")
df.head()

Dimensiones del dataset original: (3052, 10)

Columnas originales: ['Symbol', 'Name', 'Price (Intraday)', 'Change', '% Change', 'Market Cap', 'Volume in Currency (Since 0:00 UTC)', 'Volume in Currency (24Hr)', 'Total Volume All Currencies (24Hr)', 'Circulating Supply']

Primeras 5 filas:


,Symbol,Name,Price (Intraday),Change,% Change,Market Cap,Volume in Currency (Since 0:00 UTC),Volume in Currency (24Hr),Total Volume All Currencies (24Hr),Circulating Supply
0,BTC-USD,Bitcoin USD,"27,921.96",-101.1700,-0.36%,540.056B,10.068B,10.068B,10.068B,19.342M
1,ETH-USD,Ethereum USD,"1,838.81",-29.0100,-1.55%,221.499B,5.859B,5.859B,5.859B,120.458M
2,USDT-USD,Tether USD,1.0004,0.0000,0.00%,80.240B,18.796B,18.796B,18.796B,80.211B
3,BNB-USD,BNB USD,310.8,-1.5500,-0.50%,49.072B,460.394M,460.394M,460.394M,157.887M
4,USDC-USD,USD Coin USD,0.9997,-0.0001,-0.01%,32.592B,2.389B,2.389B,2.389B,32.6B


## 2. Crear una Copia para Ensuciar

In [ ]:
# Crear una copia para ensuciar
dirty_df = df.copy()
print(f"Trabajando con una copia del dataset: {dirty_df.shape}")

Trabajando con una copia del dataset: (3052, 10)


## 3. Tipo de Error 1: Datos Faltantes (Valores NaN aleatorios)

In [ ]:
# Introducir valores faltantes aleatoriamente en diferentes columnas
columns_to_nan = ['Price (Intraday)', 'Market Cap', 'Volume in Currency (Since 0:00 UTC)', 'Circulating Supply']

for col in columns_to_nan:
    if col in dirty_df.columns:
        # Seleccionar aleatoriamente 5-10% de filas para establecer como NaN
        nan_indices = np.random.choice(dirty_df.index, size=int(len(dirty_df) * 0.07), replace=False)
        dirty_df.loc[nan_indices, col] = np.nan

print(f"Introducidos valores faltantes en {len(columns_to_nan)} columnas")
print(f"Valores faltantes por columna:\n{dirty_df.isnull().sum()[dirty_df.isnull().sum() > 0]}")

Introducidos valores faltantes en 4 columnas
Valores faltantes por columna:
Price (Intraday)                       213
Market Cap                             213
Volume in Currency (Since 0:00 UTC)    213
Circulating Supply                     213
dtype: int64


## 4. Tipo de Error 2: Filas Duplicadas

In [ ]:
# Añadir duplicados exactos
duplicate_indices = np.random.choice(dirty_df.index, size=50, replace=False)
duplicates = dirty_df.loc[duplicate_indices].copy()
dirty_df = pd.concat([dirty_df, duplicates], ignore_index=True)

# Añadir duplicados parciales (mismo símbolo, datos diferentes)
partial_dup_indices = np.random.choice(dirty_df.index, size=30, replace=False)
partial_duplicates = dirty_df.loc[partial_dup_indices].copy()
# Modificar algunos valores para hacerlos duplicados parciales
if 'Price (Intraday)' in partial_duplicates.columns:
    partial_duplicates['Price (Intraday)'] = partial_duplicates['Price (Intraday)'].apply(
        lambda x: str(float(str(x).replace(',', '')) * 1.1) if pd.notna(x) and x != '' else x
    )
dirty_df = pd.concat([dirty_df, partial_duplicates], ignore_index=True)

print(f"Añadidos 50 duplicados exactos y 30 duplicados parciales")
print(f"Nuevas dimensiones del dataset: {dirty_df.shape}")

Añadidos 50 duplicados exactos y 30 duplicados parciales
Nuevas dimensiones del dataset: (3132, 10)


## 5. Tipo de Error 3: Valores Atípicos (Valores Extremos)

In [ ]:
# Introducir valores atípicos en columnas numéricas
outlier_indices = np.random.choice(dirty_df.index, size=40, replace=False)

for idx in outlier_indices:
    if 'Price (Intraday)' in dirty_df.columns and pd.notna(dirty_df.loc[idx, 'Price (Intraday)']):
        # Hacer el precio extremadamente alto o bajo
        if random.random() > 0.5:
            dirty_df.loc[idx, 'Price (Intraday)'] = '999999999.99'
        else:
            dirty_df.loc[idx, 'Price (Intraday)'] = '0.000000001'

print(f"Introducidos {len(outlier_indices)} valores atípicos en la columna Price")

Introducidos 40 valores atípicos en la columna Price


## 6. Tipo de Error 4 y 10: Inconsistencias de Formato y Puntuación Extra

In [ ]:
# Mezclar diferentes formatos numéricos y añadir puntuación extra
format_indices = np.random.choice(dirty_df.index, size=100, replace=False)

for idx in format_indices:
    # Añadir símbolos de moneda
    if 'Price (Intraday)' in dirty_df.columns and pd.notna(dirty_df.loc[idx, 'Price (Intraday)']):
        price = str(dirty_df.loc[idx, 'Price (Intraday)'])
        dirty_df.loc[idx, 'Price (Intraday)'] = f"${price}" if random.random() > 0.5 else f"{price}€"
    
    # Añadir comas o puntos extra
    if 'Market Cap' in dirty_df.columns and pd.notna(dirty_df.loc[idx, 'Market Cap']):
        mcap = str(dirty_df.loc[idx, 'Market Cap'])
        if 'B' in mcap or 'M' in mcap:
            dirty_df.loc[idx, 'Market Cap'] = mcap.replace('.', ',') if random.random() > 0.5 else mcap

print(f"Introducidas inconsistencias de formato y puntuación extra en {len(format_indices)} filas")

Introducidas inconsistencias de formato y puntuación extra en 100 filas


## 7. Tipo de Error 5: Errores Tipográficos

In [ ]:
# Introducir errores tipográficos en la columna Name
def introduce_typo(text):
    if pd.isna(text) or len(str(text)) < 3:
        return text
    text = str(text)
    typo_type = random.choice(['swap', 'delete', 'duplicate', 'replace'])
    pos = random.randint(1, len(text) - 2)
    
    if typo_type == 'swap' and pos < len(text) - 1:
        text = text[:pos] + text[pos+1] + text[pos] + text[pos+2:]
    elif typo_type == 'delete':
        text = text[:pos] + text[pos+1:]
    elif typo_type == 'duplicate':
        text = text[:pos] + text[pos] + text[pos:]
    elif typo_type == 'replace':
        text = text[:pos] + random.choice(string.ascii_lowercase) + text[pos+1:]
    
    return text

typo_indices = np.random.choice(dirty_df.index, size=80, replace=False)
if 'Name' in dirty_df.columns:
    for idx in typo_indices:
        dirty_df.loc[idx, 'Name'] = introduce_typo(dirty_df.loc[idx, 'Name'])

print(f"Introducidos errores tipográficos en {len(typo_indices)} nombres de criptomonedas")

Introducidos errores tipográficos en 80 nombres de criptomonedas


## 8. Tipo de Error 6: Categorías Extra

In [ ]:
# Añadir valores inusuales/incorrectos en la columna Symbol
unusual_symbols = ['XXX-USD', 'DESCONOCIDO', 'N/A', 'POR_DEFINIR', '???', 'ERROR', 'NULO']
unusual_indices = np.random.choice(dirty_df.index, size=25, replace=False)

if 'Symbol' in dirty_df.columns:
    for idx in unusual_indices:
        dirty_df.loc[idx, 'Symbol'] = random.choice(unusual_symbols)

print(f"Añadidos {len(unusual_indices)} valores de categoría inusuales")

Añadidos 25 valores de categoría inusuales


## 9. Tipo de Error 7: Tipos de Datos Incorrectos

In [ ]:
# Convertir algunas columnas numéricas a texto
text_indices = np.random.choice(dirty_df.index, size=50, replace=False)

for idx in text_indices:
    if '% Change' in dirty_df.columns and pd.notna(dirty_df.loc[idx, '% Change']):
        dirty_df.loc[idx, '% Change'] = f"aproximadamente {dirty_df.loc[idx, '% Change']}"

print(f"Convertidos {len(text_indices)} valores numéricos a cadenas de texto")

Convertidos 50 valores numéricos a cadenas de texto


## 10. Tipo de Error 8: Problemas de Codificación No-UTF-8

In [ ]:
# Añadir caracteres especiales y problemas de codificación
special_chars = ['©', '®', '™', '€', '£', '¥', 'Ã', 'Ñ', 'ñ', 'ü', 'ö']
encoding_indices = np.random.choice(dirty_df.index, size=40, replace=False)

if 'Name' in dirty_df.columns:
    for idx in encoding_indices:
        if pd.notna(dirty_df.loc[idx, 'Name']):
            name = str(dirty_df.loc[idx, 'Name'])
            pos = random.randint(0, len(name))
            dirty_df.loc[idx, 'Name'] = name[:pos] + random.choice(special_chars) + name[pos:]

print(f"Añadidos caracteres especiales en {len(encoding_indices)} nombres")

Añadidos caracteres especiales en 40 nombres


## 11. Tipo de Error 9: Encabezados Incorrectos

In [ ]:
# Renombrar algunas columnas con errores tipográficos o nombres incorrectos
column_mapping = {
    'Symbol': 'Simbolo',  # Traducción
    'Name': 'NombreCripto',  # Nombre diferente
    '% Change': 'PorcentajeCambio',  # Eliminar carácter especial
    'Market Cap': 'CapitalizacionMercado',  # Eliminar espacio
}

# Solo renombrar columnas que existen
rename_dict = {k: v for k, v in column_mapping.items() if k in dirty_df.columns}
dirty_df = dirty_df.rename(columns=rename_dict)

print(f'Renombrados {len(rename_dict)} encabezados de columna')
print(f'Nuevas columnas: {dirty_df.columns.tolist()}')

Renombrados 4 encabezados de columna
Nuevas columnas: ['Simbolo', 'NombreCripto', 'Price (Intraday)', 'Change', 'PorcentajeCambio', 'CapitalizacionMercado', 'Volume in Currency (Since 0:00 UTC)', 'Volume in Currency (24Hr)', 'Total Volume All Currencies (24Hr)', 'Circulating Supply']


## 12. Tipo de Error 11: Capitalización Inconsistente

In [ ]:
# Mezclar capitalización en columnas de texto
def randomize_case(text):
    if pd.isna(text):
        return text
    text = str(text)
    case_type = random.choice(['upper', 'lower', 'title', 'mixed'])
    
    if case_type == 'upper':
        return text.upper()
    elif case_type == 'lower':
        return text.lower()
    elif case_type == 'title':
        return text.title()
    else:  # mixed
        return ''.join(c.upper() if random.random() > 0.5 else c.lower() for c in text)

case_indices = np.random.choice(dirty_df.index, size=100, replace=False)
if 'NombreCripto' in dirty_df.columns:
    for idx in case_indices:
        dirty_df.loc[idx, 'NombreCripto'] = randomize_case(dirty_df.loc[idx, 'NombreCripto'])

print(f"Capitalización aleatoria en {len(case_indices)} nombres")

Capitalización aleatoria en 100 nombres


## 13. Tipo de Error 12: Problemas de Espacios en Blanco

In [ ]:
# Añadir espacios al inicio/final y espacios extra
def add_whitespace(text):
    if pd.isna(text):
        return text
    text = str(text)
    ws_type = random.choice(['leading', 'trailing', 'both', 'internal'])
    
    if ws_type == 'leading':
        return '  ' + text
    elif ws_type == 'trailing':
        return text + '  '
    elif ws_type == 'both':
        return '  ' + text + '  '
    else:  # internal
        words = text.split()
        return '  '.join(words)

ws_indices = np.random.choice(dirty_df.index, size=120, replace=False)
for idx in ws_indices:
    if 'Simbolo' in dirty_df.columns:
        dirty_df.loc[idx, 'Simbolo'] = add_whitespace(dirty_df.loc[idx, 'Simbolo'])
    if 'NombreCripto' in dirty_df.columns and random.random() > 0.5:
        dirty_df.loc[idx, 'NombreCripto'] = add_whitespace(dirty_df.loc[idx, 'NombreCripto'])

print(f"Añadidos problemas de espacios en {len(ws_indices)} filas")

Añadidos problemas de espacios en 120 filas


## 14. Resumen de Todos los Errores Introducidos

In [ ]:

print('RESUMEN DE ERRORES INTRODUCIDOS')

print(f'\n1. Datos Faltantes: ~7% en 4 columnas')
print(f'2. Filas Duplicadas: 50 exactos + 30 duplicados parciales')
print(f'3. Valores Atípicos: 40 valores extremos en Price')
print(f'4. Inconsistencias de Formato: 100 filas con formatos mixtos')
print(f'5. Errores Tipográficos: 80 errores en nombres')
print(f'6. Categorías Extra: 25 valores de símbolo inusuales')
print(f'7. Tipos de Datos Incorrectos: 50 números como texto')
print(f'8. Codificación No-UTF-8: 40 caracteres especiales')
print(f'9. Encabezados Incorrectos: 4 nombres de columna cambiados')
print(f'10. Puntuación Extra: 100 filas con símbolos de moneda')
print(f'11. Capitalización Inconsistente: 100 nombres')
print(f'12. Problemas de Espacios: 120 filas')
print(f'\nDimensiones originales: {df.shape}')
print(f'Dimensiones dataset sucio: {dirty_df.shape}')
print(f'\nInformación del dataset sucio:')
print(dirty_df.info())

RESUMEN DE ERRORES INTRODUCIDOS

1. Datos Faltantes: ~7% en 4 columnas
2. Filas Duplicadas: 50 exactos + 30 duplicados parciales
3. Valores Atípicos: 40 valores extremos en Price
4. Inconsistencias de Formato: 100 filas con formatos mixtos
5. Errores Tipográficos: 80 errores en nombres
6. Categorías Extra: 25 valores de símbolo inusuales
7. Tipos de Datos Incorrectos: 50 números como texto
8. Codificación No-UTF-8: 40 caracteres especiales
9. Encabezados Incorrectos: 4 nombres de columna cambiados
10. Puntuación Extra: 100 filas con símbolos de moneda
11. Capitalización Inconsistente: 100 nombres
12. Problemas de Espacios: 120 filas

Dimensiones originales: (3052, 10)
Dimensiones dataset sucio: (3132, 10)

Información del dataset sucio:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3132 entries, 0 to 3131
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Simbolo  

## 15. Guardar Dataset Sucio

In [ ]:
# Guardar con problemas de codificación (no UTF-8)
dirty_df.to_csv('dirty_dataset.csv', index=False, encoding='latin-1', errors='replace')

print("Dataset sucio guardado como 'dirty_dataset.csv' con codificación latin-1")
print(f"\nDimensiones finales del dataset: {dirty_df.shape}")
print(f"\nPrimeras 10 filas del dataset sucio:")
dirty_df.head(10)

Dataset sucio guardado como 'dirty_dataset.csv' con codificación latin-1

Dimensiones finales del dataset: (3132, 10)

Primeras 10 filas del dataset sucio:


,Simbolo,NombreCripto,Price (Intraday),Change,PorcentajeCambio,CapitalizacionMercado,Volume in Currency (Since 0:00 UTC),Volume in Currency (24Hr),Total Volume All Currencies (24Hr),Circulating Supply
0,BTC-USD,Bitcoin USD,NaN,-101.170000,-0.36%,540.056B,10.068B,10.068B,10.068B,19.342M
1,ETH-USD,Ethereum USD,"1,838.81",-29.010000,-1.55%,NaN,5.859B,5.859B,5.859B,120.458M
2,USDT-USD,Tether USD,$1.0004,0.000000,0.00%,"80,240B",NaN,18.796B,18.796B,80.211B
3,BNB-USD,BNB USD,310.8,-1.550000,-0.50%,49.072B,NaN,460.394M,460.394M,157.887M
4,USDC-USD,USD zo¥in USD,0.9997,-0.000100,-0.01%,32.592B,2.389B,2.389B,2.389B,32.6B
5,XRP-USD,XRP USD,0.502372,-0.007247,-1.42%,25.967B,565.59M,565.59M,565.59M,51.688B
6,ADA-USD,Cardano USD,0.3888,0.001100,0.002807,13.517B,197.578M,197.578M,197.578M,34.762B
7,HEX-USD,HEX USD,0.0711,0.001100,0.015203,12.336B,6.016M,6.016M,6.016M,173.411B
8,DOGE-USD,Dogecoin USD,0.0834,0.000200,0.25%,11.582B,489.325M,489.325M,489.325M,NaN
9,DESCONOCIDO,Lido Staked ETH USD,"1,820.71",-35.330000,-1.90%,10.798B,44.878M,44.878M,44.878M,5.931M


## 16. Verificación - Mostrar Ejemplos de Errores

In [ ]:
# Mostrar ejemplos de cada tipo de error
print('EJEMPLOS DE ERRORES:\n')

# Valores faltantes
print('1. Valores Faltantes:')
print(dirty_df[dirty_df['Price (Intraday)'].isna()].head(3))

# Duplicados
print('\n2. Duplicados:')
print(f'Total de duplicados: {dirty_df.duplicated().sum()}')

# Valores atípicos
print('\n3. Valores Atípicos (precios extremos):')
print(dirty_df[dirty_df['Price (Intraday)'].astype(str).str.contains('999999', na=False)].head(2))

# Problemas de formato
print('\n4. Problemas de Formato (símbolos de moneda):')
print(dirty_df[dirty_df['Price (Intraday)'].astype(str).str.contains('[$€]', na=False)].head(3))

# Espacios en blanco
print('\n5. Problemas de Espacios:')
print(dirty_df[dirty_df['Simbolo'].astype(str).str.contains('^\\s|\\s$', na=False)].head(3))

print('\n')
print('¡ENSHITIFICACIÓN DE DATOS COMPLETADA!')


EJEMPLOS DE ERRORES:

1. Valores Faltantes:
      Simbolo      NombreCripto Price (Intraday)      Change PorcentajeCambio  \
0     BTC-USD       Bitcoin USD              NaN -101.170000           -0.36%   
14   WTRX-USD  Wrapped TRON USD              NaN   -0.000374           -0.56%   
32    FIL-USD    filecoin usd                NaN   -0.158500           -2.72%   

   CapitalizacionMercado Volume in Currency (Since 0:00 UTC)  \
0               540.056B                             10.068B   
14                6.716B                             307,261   
32                2.343B                                 NaN   

   Volume in Currency (24Hr) Total Volume All Currencies (24Hr)  \
0                    10.068B                            10.068B   
14                   307,261                            307,261   
32                  159.354M                           159.354M   

   Circulating Supply  
0             19.342M  
14           101.676B  
32           413.576M  

2. Dupli